In [ ]:
#default_exp index

# Index

> Tools to retrieve the PDS index for Cassini ISS image data.

In [ ]:
#export
from pathlib import Path

import numpy as np
import pandas as pd

from planetarypy.pdstools.indexes import indexdb

from pyciss.ringutils import which_epi_janus_resonance, get_all_prime_resonances
from pyciss.io import config

In [ ]:
#export
resonances = get_all_prime_resonances()

def get_index_dir():
    d = Path(config.archive_path) / "indexes"
    d.mkdir(exist_ok=True)
    return d

In [ ]:
get_index_dir()

PosixPath('/home/maye/big_drive/planetary_data/pyciss/indexes')

In [ ]:
#export
def download_general_index():
    indexdb.download("cassini.iss.index")

In [ ]:
#export
def download_ring_summary_index():
    indexdb.download("cassini.iss.ring_summary")

In [ ]:
#export
def read_cumulative_iss_index():
    "Read in the whole cumulative index and return dataframe."
    index = indexdb.get("cassini.iss.index")

    df = index.df
    # replace PDS Nan values (-1e32) with real NaNs
    df = df.replace(-1.000000e32, np.nan)
    return df.replace(-999.0, np.nan)

In [ ]:
#export
def ring_summary_index():
    index = indexdb.get('cassini.iss.ring_summary')

    df = index.df
    df = df.replace(-1.00000e32, np.nan)
    return df.replace(-999.0, np.nan)

In [ ]:
df = ring_summary_index()

In [ ]:
df.head()

,VOLUME_ID,FILE_SPECIFICATION_NAME,OPUS_ID,MINIMUM_RIGHT_ASCENSION,MAXIMUM_RIGHT_ASCENSION,MINIMUM_DECLINATION,MAXIMUM_DECLINATION,MINIMUM_RING_RADIUS,MAXIMUM_RING_RADIUS,FINEST_RING_INTERCEPT_RESOLUTION,...,RING_CENTER_DISTANCE,SUB_SOLAR_RING_LONGITUDE,SUB_OBSERVER_RING_LONGITUDE,RING_CENTER_PHASE_ANGLE,RING_CENTER_INCIDENCE_ANGLE,RING_CENTER_NORTH_BASED_INCIDENCE_ANGLE,RING_CENTER_EMISSION_ANGLE,RING_CENTER_NORTH_BASED_EMISSION_ANGLE,SOLAR_RING_OPENING_ANGLE,OBSERVER_RING_OPENING_ANGLE
0,COISS_2001,data/1454725799_1455008789/N1454725799_1.LBL,co-iss-n1454725799,35.528398,35.905555,9.719637,10.091212,205899.335,1098621.586,419.01370,...,7.087500e+07,284.523,215.951,64.070,64.589,115.411,73.676,106.324,-25.411,-16.324
1,COISS_2001,data/1454725799_1455008789/N1454726579_1.LBL,co-iss-n1454726579,36.662898,37.039038,9.590114,9.960833,721136.155,1745689.307,421.97145,...,7.087085e+07,284.524,215.951,64.070,64.589,115.411,73.676,106.324,-25.411,-16.324
2,COISS_2001,data/1454725799_1455008789/N1454727359_1.LBL,co-iss-n1454727359,36.928066,37.304334,9.879593,10.250110,1042341.438,2186830.920,414.79041,...,7.086670e+07,284.524,215.951,64.071,64.589,115.411,73.676,106.324,-25.411,-16.324
3,COISS_2001,data/1454725799_1455008789/N1454728139_1.LBL,co-iss-n1454728139,28.100420,28.479618,6.224709,6.601574,NaN,NaN,NaN,...,7.086256e+07,284.524,215.951,64.071,64.589,115.411,73.676,106.324,-25.411,-16.324
4,COISS_2001,data/1454725799_1455008789/N1454728919_1.LBL,co-iss-n1454728919,37.981418,38.357791,10.611947,10.981697,3865717.589,3905151.343,405.91234,...,7.085841e+07,284.525,215.951,64.071,64.589,115.411,73.676,106.324,-25.411,-16.324


In [ ]:
#export
def read_ring_images_index():
    """Filter cumulative index for ring images.

    This is done by matching the column TARGET_DESC to contain the string 'ring'

    Returns
    -------
    pandas.DataFrame
        data table containing only meta-data for ring images
    """
    meta = read_cumulative_iss_index()
    ringfilter = meta.TARGET_DESC.str.contains("ring", case=False)
    return meta[ringfilter]

In [ ]:
read_ring_images_index().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117692 entries, 2321 to 407222
Columns: 140 entries, FILE_NAME to STANDARD_DATA_PRODUCT_ID
dtypes: datetime64[ns](7), float64(70), int64(20), object(43)
memory usage: 126.6+ MB


In [ ]:
#export
def get_clearnacs_ring_images():
    df = read_ring_images_index()
    try:
        df = df.set_index("isotime")
    except KeyError:
        print("'isotime' column does not exist. Leaving index as it is.")
    ringimages = df.query("RINGS_FLAG=='YES'")
    ringimages = ringimages[ringimages.MAXIMUM_RING_RADIUS.notnull()]
    ringimages = ringimages[ringimages.MINIMUM_RING_RADIUS.notnull()]
    ringimages = ringimages.query(
        "MAXIMUM_RING_RADIUS < 1e90 and MINIMUM_RING_RADIUS > 0"
    )
    nac = ringimages[ringimages.INSTRUMENT_ID == "ISSNA"]
    clearnacs = nac.query('FILTER_NAME_1 == "CL1" and FILTER_NAME_2 == "CL2"')
    return clearnacs

In [ ]:
clearnacs = get_clearnacs_ring_images()
clearnacs.info()

'isotime' column does not exist. Leaving index as it is.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 73271 entries, 2321 to 407222
Columns: 140 entries, FILE_NAME to STANDARD_DATA_PRODUCT_ID
dtypes: datetime64[ns](7), float64(70), int64(20), object(43)
memory usage: 78.8+ MB


In [ ]:
#export
def filter_for_ringspan(clearnacs, spanlimit):
    "filter for covered ringspan, giver in km."
    delta = clearnacs.MAXIMUM_RING_RADIUS - clearnacs.MINIMUM_RING_RADIUS
    f = delta < spanlimit
    ringspan = clearnacs[f].copy()
    return ringspan

In [ ]:
filter_for_ringspan(clearnacs, 500).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134 entries, 13326 to 398037
Columns: 140 entries, FILE_NAME to STANDARD_DATA_PRODUCT_ID
dtypes: datetime64[ns](7), float64(70), int64(20), object(43)
memory usage: 147.6+ KB


In [ ]:
#export
def get_resonances_inside_radius(row):
    minrad = row["MINIMUM_RING_RADIUS"]
    maxrad = row["MAXIMUM_RING_RADIUS"]
    lower_filter = resonances["radius"] > (minrad)
    higher_filter = resonances["radius"] < (maxrad)
    insides = resonances[lower_filter & higher_filter]
    return insides


def check_for_resonance(row, as_bool=True):
    insides = get_resonances_inside_radius(row)
    return bool(len(insides)) if as_bool else len(insides)


def check_for_janus_resonance(row, as_bool=True):
    insides = get_resonances_inside_radius(row)
    # row.name is the index of the row, which is a time!
    janus = which_epi_janus_resonance("janus", row.name)
    moonfilter = insides.moon == janus
    return bool(len(insides[moonfilter]))


def get_janus_phase(time):
    return which_epi_janus_resonance("janus", time)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_io.ipynb.
Converted 01_opusapi.ipynb.
Converted 02_pipeline.ipynb.
Converted 03_ringutils.ipynb.
Converted 04_plotting.ipynb.
Converted 05_index.ipynb.
Converted 06_ringcube.ipynb.
Converted index.ipynb.
Converted to_be_implemented.ipynb.
